# 3W Notebook

<style>
td, th {
   border: none!important;
}
</style>

| Title         	| Description                                                                                                                                                                                      	|
|---------------	|--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------	|
| Script        	| 3w_Core_Notebook.ipynb                                                                                                                                                                           	|
| Description   	| Create the geopackage use to label 3w maps                                                                                                                                                       	|
| Author        	| Tony Giless  <br>LF Velasquez<br>Darren Connaghan<br>Ant Scott<br>                                                                                                                                	|
| Date          	| Mar 2024                                                                                                                                                                                         	|
| Version       	| 1.3.1                                                                                                                                                                                              	|
| Usage         	| This notebook can be run using the arcpro environemt or inside ArcPro the later is the recommended way                                                                                           	|
| Notes         	| - Please make sure to edit the cell that is expected user input. Otherwise, the output will no reflect the right information <br>- The laptop needs to have the Humanitarian Icon font installed <br>- This new version add the label expresion for QGIS 	|
| Pytho version 	| 3.7.10 - Deployed with ArcPro                                                                                                                                                                    	|

________________________________________________________________
________________________________________________________________

### 1. Setting packages
These packages i.e. geopandas, numpy etc are already installed in arcpro. There is no need to do anything else apart from running the cell

In [ ]:
import os
os.environ['USE_PYGEOS'] = '0' # This avoids a projection warning. It is down to the ArcPro set up
import sys
import pip
import subprocess

import pandas as pd
import arcpy
import numpy as np
from pathlib import Path
from arcgis.features import GeoAccessor, GeoSeriesAccessor
import datetime
from pprint import pprint

# This is just for aesthetic purposes when printing the dataframes
pd.set_option('display.max_colwidth', None)

### 2. User Variables

<div class="alert alert-block alert-info">
<b>Tip:</b> The cell below has the variables that need to be edited by the user. Please pay attention to the format of the variables and make sure you replicate them correctly with the information you need<br>
<b>Tip:</b> The user will only need to set this once at the start of the process. After the initial set up this notebook can be run as many times as needed</div>

<div class="alert alert-block alert-warning">
<b>Warning:</b> The names of the sectors / clusters need to be the same in the 3w data and the iconDict that includes the language i.e. if the data is in Spanish then the keys in the iconDict need to be in Spanish. The dictionary can also have as many names as needed, just make sure you add the code for the icon. </div>

In [ ]:
################################################################
# Main path
################################################################

# Main Path
'''As this is running in ArcPro the Path().resolve read the Path based in the location of the map project.
To take it to the GIS foder the parent methods needs to be used'''

# src_path = Path().resolve().parent.parent.parent # Use this line if running inside ArcPro
# src_path = Path().resolve().parent.parent # Use this line if running outside ArcPro
src_path = Path().resolve().parent.parent # Use this line if running outside ArcPro



################################################################
################################################################

################################################################
# Sector / Cluster
################################################################
'''
Icon Dictionary
This is the dictionary containing the sector/cluster names and the 
icon unicode. Make sure the sector/cluster name match what you have
in the 3w spreadsheet, this includes also the language. If you need
to know the icon for a sector/cluster please check the icon lookup table: 
https://github.com/mapaction/ocha-humanitarian-icons-for-gis/blob/master/documentation/icon-lookup-table.md
'''
# English Dictionary
# Icon dictionary - add any sectors that may be required
iconDict = {
     'Cash': u'\ue930',
     'Shelter and NFI': u'\ue92c',
     'Education' : u'\ue95d',
     'Food Security': u'\ue973',
     'Health': u'\ue981',
     'Logistics': u'\ue9b6',
     'Protection': u'\ue9fa',
     'WASH': u'\uea61',
     'Coordination': u'\ue945',
     'Early Recovery and Livelihoods': u'\ue95a',
     'Nutrition': u'\ue945',
     }

# # Pportuguese Dictionary
# iconDict = {
#     'Transferência de dinheiro': u'\ue930', # This can also be 'CASH'
#     'Abrigo/ Artigos não alimentares': u'\ue92c', # This can also be 'Abrigo e NFI' or 'Abrigo e itens não alimentares
#     'Educação' : u'\ue95d',
#     'Segurança Alimentar e Meios de Subsistência': u'\ue973',
#     'Saúde': u'\ue981',
#     'Proteção': u'\ue9fa',
#     'Água, Saneamento e Higiene': u'\uea61', # This can also be defined as 'WASH'
#     'Logística': u'\ue9b6',
#     'Nutrição': u'\ue945',
#     'Coordenação e gerenciamento de acampamento': u'\ue945', # This can also be defined as 'CCCM'
#     'Recuperação precoce e meios de subsistência': u'\ue95a',
#     'Coordenação': u'\ue945'
#     }

################################################################
################################################################

################################################################
# Shapefile and Excel set up
################################################################
'''
In this section the user will set the name of the shapefile to be used
e.g. (admn1 or admn2 or admn3), this can be eiter point or polygon. As well as the
name of the spreadsheet tabs and columns to be used as part of the script
'''

# Setting the admin polygon shapefile name
admin_shp = 'gha_admn_ad2_py_s1_gss_pp_districts_em.shp' # name of the shapefile and extension

# Setting 3w data excel file
excel_file = 'gha_www_datafixed_DC.xlsx' # name of the excel file and extension
main_tab = '3w_all_data'  # this is the name of the tab with the main data
acron_tab = 'Acronym'     # this is the name of the tab with the acronym data

# Select how you want the data needs to be grouped and processed
'''
If creating 3w for admn 2 then you need to enter the admn2 fields
'''
admn_group = 'ADM2_EN'     # this is how the ADMN column is named in the Excel file
pcod_group = 'ADM2_PCODE'  # this is how the PCODE column is named in the Excel file
admn_ref = 2               # this needs to be the reference to the admn boundary e.g. 1 for ADMN1 

# Setting Sector, Status and Organization columns
SECTOR = 'Sector'          # this is how the sector/cluster column is named in the Excel file
STATUS = 'Status'          # this is how the status column is named in the Excel file
ORGS = 'Organization Name' # this is how the organization column is named in the Excel file

# Setting type of activity
ongoing = 'Ongoing'        # this is how the task is defined in the Excel file 
planned = 'Planned'        # this is how the task is defined in the Excel file 

# Output type
'''
Please make sure you enter what you would like to have as an output
fc = feature class inside a 3WWW database insidde the 201_3www folder
gpkg = geopckage inside the 201_3www folder
'''
OUT_TYPE = 'gpkg'

print('----------------------------\n'
      'User variables have been set\n'
      '----------------------------')

#### Functions

In [ ]:
def check_var_setup(path_src, xlsx_file, dict_icon):
    """Check that the iconDict and activites status has been 
    set correclty

    Args:
        path_src (str): path to main folder
        xlsx_file (str): name of the excel file including extension
        dict_icon (dict): dictionary containing sector and unicode
    """
    
    # Read Excel file with 3W data
    df_data = pd.read_excel(Path(path_src / f'2_Active_Data/201_3www/{xlsx_file}'), index_col=None, sheet_name=main_tab)
    
    # Create list of sector - unique values
    unique_sector = list(set(df_data[SECTOR].to_list()))
    
    # Create list of action status - unique value
    unique_status = list(set(df_data[STATUS].to_list()))
    
    # Remove nan if exist - that is an empty sector value
    unique_sector = [item for item in unique_sector if not(pd.isnull(item)) == True]
    unique_status = [item for item in unique_status if not(pd.isnull(item)) == True]

    # Check that sectors in excel file are listed in the dictIcon
    missing_sector = list(unique_sector - dict_icon.keys())
    
    if len(missing_sector) != 0:
        print ('there is an error in iconDict.\n' 
               f'The following values are in the Excel file but not in the iconDict\n'
               '-----------------------------------------------\n')
        pprint(missing_sector)
        print('\n-----------------------------------------------\n'
              'Please fix the iconDict before continuing\n'
              '-----------------------------------------------')
    else:
        print('----------------------------------------------------\n'
              'iconDict set correctly\n'
              '----------------------------------------------------')
    
    # Check if action status variables have been defined correctly
    lst_status = [ongoing,planned]
    missing_status = all(element in unique_status for element in lst_status)
    if missing_status:
        print('----------------------------------------------------\n'
              'Activites status set correctly\n'
              '----------------------------------------------------')
    else:
        print ('There is an error in the activity status.\n' 
               f'\nHere are the values in the Excel file:\n')
        pprint(unique_status)
        print (f'\nHere are the values you defined:\n')
        pprint(lst_status)
        print('\n-----------------------------------------------\n'
              'Please correct any error you see in spelling\n'
              '-----------------------------------------------')

def df_replace(dataframe, col_name, value_to_replace, new_value):
    """Replacing names in columns 

    Args:
        dataframe (pandas.dataframe): pandas dataframe with the data
        col_name (str): name of the column to be searched
        value_to_replace (str): value to be found in the column
        new_value (str): new value

    Returns:
        pandas.dataframe: pandas dataframe with the correct names in the column entered
    """
    dataframe.loc[ dataframe[col_name] == value_to_replace, col_name] = new_value
    return dataframe

def remove_empty(df_to_change, column_name):
    """remove rows where value in column is NaN

    Args:
        df_to_change (pandas.dataframe): dataframe containing the data to be modified
        column_name (str): column name to be checked in the df_to_change

    Returns:
        pandas.dataframe: subset of the dataframe removing rows with NaN values in column
    """
    df_to_change[column_name].replace('', np.nan, inplace=True)
    df_to_change = df_to_change[df_to_change[column_name].notna()]
    return df_to_change

# Function to add break in organization
def add_break(row):
    """Set the organization level in the format required.
    That is, every forth organization adds a break

    Args:
        row (pandas.series): panda dataframe row

    Returns:
        str: string of organizations with breaks added
    """
    lst_names = row['organisations'].split(',')
    n = 4
    if len(lst_names) > 4:
        for i in range ( n, len(lst_names), n):
            lst_names[i] = '\n' + lst_names[i]
        return ",".join(lst_names)
    else:
        return row['organisations']

# Function to build the label
def label_create(row, sector_lbl,
                 close_act, total_lbl,
                 inpro_lbl, plan_lbl):
    """This function creates a label for each string
    using the information on each row of the dataframe.

    Args:
        row (pandas.series): panda dataframe row
        sector_lbl (str): string for format of the sector label
        close_act (str): string for format the end part of the activity label
        total_lbl (str): string for format the start part of the total activity label
        inpro_lbl (str): string for format the start part of the in progress activity label
        plan_lbl (str): string for format the start part of the planned activity label
    """

    return(f'{sector_lbl} {iconDict[row["sector"]]} {close_act} \t'
        f'{total_lbl} {row["total_activites"]} {close_act} \t'
        f'{inpro_lbl} {row["total_inProg"]} {close_act} \t'
        f'{plan_lbl} {row["total_planned"]} {close_act} \t'
        f'<BOL>{row["organisations"]}</BOL>'
        )

def label_create_qgis(row, sector_lbl,
                 close_act, total_lbl,
                 inpro_lbl, plan_lbl):
    """This function creates a label for each string
    using the information on each row of the dataframe.

    Args:
        row (pandas.series): panda dataframe row
        sector_lbl (str): string for format of the sector label
        close_act (str): string for format the end part of the activity label
        total_lbl (str): string for format the start part of the total activity label
        inpro_lbl (str): string for format the start part of the in progress activity label
        plan_lbl (str): string for format the start part of the planned activity label
    """
    join_clause = "||"
    
    final_lbl = (f'{sector_lbl} {iconDict[row["sector"]]}{close_act}'
        f'\t{total_lbl}{join_clause}{row["total_activites"]}{join_clause}' + f"'{close_act}"
        f'\t{inpro_lbl}{join_clause}{row["total_inProg"]}{join_clause}' + f"'{close_act}"
        f'\t{plan_lbl}{join_clause}{row["total_planned"]}{join_clause}' + "'"
        f'''\t{close_act}<b>{row["organisations"]}</b>{close_act}''' + "</p>'"
        )

    return(final_lbl)

# Function to create the label for ArcPro
def label_gis(row, col_name, software):
    """ Unpack the label list ready for ArcPro

    Args:
        row (pandas.series): panda dataframe row

    Returns:
        str: the string to be used for the label in ArcPro
    """
    # Select list with the label
    lst_lbl = row[col_name]
    
    # If ADM name is too long, the split into several lines
    adm_lbl = row[admn_group].upper().split(" ")
    n = 3
    if len(adm_lbl) > 3:
        for i in range ( n, len(adm_lbl), n):
            adm_lbl[i] = '\n' + adm_lbl[i]
            adm_lbl = " ".join(adm_lbl)
    else:
        adm_lbl = " ".join(adm_lbl)
    
    
    # Create a single string from the list - this is the label
    if software.lower() == 'arcpro':
        final_label = ''.join(lst_lbl)
        final_label = f"<FNT name='calibri' size='12'>"\
            f"<CLR red='0' green='0' blue='0'>"\
                f"<BOL>{adm_lbl} \t({row[pcod_group].upper()})</BOL></CLR></FNT>" + final_label
        return final_label
    
    if software.lower() == 'qgis':
        final_label = ''.join(lst_lbl)
        final_label = (''''<span style="font-family: Humanitarian-Icons;color:rgb(0,0,0)";>'''
                       f"<b>{adm_lbl} \t({row[pcod_group].upper()})</b></span>'" + final_label)
        
        # Remove extra quotation marks
        '''This is needed because we build the expression of the label 
        which then need to be modified so that it can be called from a field attribute'''
        final_label = final_label.replace("</p>''<p>","</p><p>")
        final_label = final_label.replace("'||","")
        final_label = final_label.replace("||'\t<","\t<") 
        final_label = final_label.replace("||'<","<")
        final_label = final_label.replace("</span>''<p>","</span><p>")
        final_label = final_label.replace("'<span","<span")
        final_label = final_label.replace("</p>'","</p>")
        
        return final_label
     

def gdb_3w(path_src, df_to_save, fc_name):
    """save file into a geodatbase

    Args:
        path_src (str): path to main folder
        df_to_save (dataframe): spatially enable dataframe to be saved
        fc_name (str): feature class name
    """
    gdbName = 'WWWW.gdb'
    gdbPath = str(Path(path_src / f'2_Active_Data/201_3www/{gdbName}'))
    
    # check if the database exists and add feature class         
    if arcpy.Exists(gdbPath):
        df_to_save.spatial.to_featureclass(str(Path(f'{gdbPath}/{fc_name}')))
    else:
        # Need to create database if it is the first time that runs
        arcpy.CreateFileGDB_management(str(Path(path_src / '2_Active_Data/201_3www')), gdbName)
        df_to_save.spatial.to_featureclass(str(Path(f'{gdbPath}/{fc_name}')))

def import_or_install():
    """install package if missing from env

    Args:
        package (str): name of the package to be install as per pip
    """
    try:
        import geopandas
    except ImportError:
        subprocess.check_call([sys.executable, "-m", "pip", "install", geopandas])

def saving_dataframe(shp_admin, path_src, extension, suffix, df_to_save):
    """Save excel or geopackage file from pandas or geopandas

    Args:
        shp_admin (str): name of shapefile
        path_src (str): path to main folder
        extension (str): extension of output file
        df_to_save (dataframe): pandas or geopandas dataframe

    Returns:
        file: .xlsx or .gpkg files
    """
    # Save final output
    file_name = f'{shp_admin.split("_")[0]}_3www_'\
                f'{"_".join(shp_admin.split("_")[2:-1])}_'\
                f'{shp_admin.split("_")[-1].split(".")[0]}_'\
                f'{datetime.datetime.today().strftime("%Y_%m_%d")}_{suffix}.{extension}'

    # Enter path to the www folder for the project
    www_path = Path(path_src / '2_Active_Data/201_3www/')
    
    output = Path(www_path / file_name)
    
    # save as feature class
    if extension == "fc":
        # remove extension of file name as feature class don't have extensions
        name_fc = file_name.split('.')[0]
        name_fc = name_fc.split('\\')[-1]
        gdb_3w(path_src, df_to_save, name_fc)
        return file_name
    
    # save as excel file
    if extension == "xlsx":
        df_to_save.to_excel(str(output))
        return file_name
    
    # save as geopackage
    if extension == "gpkg":
        df_to_save.to_file(str(output), driver="GPKG")
        return file_name
    else:
        return 'error'

def create_output(path_src, ref_admin, df_label, extension, field_pcode):
    """Creates feature class or geopackage to be used for the label in the map

    Args:
        path_src (str): path to main folder
        ref_admin (int): integer representing the admin boundary
        df_label (dataframe): dataframe containing the label to be used in the map
        extension (str): string of the extension i.e. gpkg or fc
        field_pcode (_type_): name of the PCODE field to be used for merge
    """
    
    if extension == 'fc':
        # set shapefile as spatial enable dataframe - this is an ArcPro type
        admin_lyr = pd.DataFrame.spatial.from_featureclass(str(Path(src_path / f'2_Active_Data/202_admn/{admin_shp}')))
    if extension != 'fc':
        # install geopandas if missing or just import geopandas
        import_or_install()
        import geopandas
        admin_lyr = geopandas.read_file(Path(src_path / f'2_Active_Data/202_admn/{admin_shp}')) # this line is needed if using geopandas
    

    # Merge label dataframe and geodataframe
    gdf_final = admin_lyr.merge(df_label, on=field_pcode)

    # # Edit dataframe to make it nice for ArcPro
    flag_x = [col for col in gdf_final.columns if '_x' in col]
    if len(flag_x) != 0:
        new_colNames = gdf_final.columns.str.rstrip('_x')
        gdf_final.columns = new_colNames

    # # Remove the label and extra admin columns as it is a list and can generate errors
    # # check if the suffix _y exists, if it does also add that to columns to be deleted
    # flag_y = [col for col in gdf_final.columns if f'{field_pcode}_y' in col]
    # if len(flag_y) != 0:
    #     gdf_final = gdf_final.drop(columns=['label', f'{field_pcode}_y'])
    # else:
    #     gdf_final = gdf_final.drop(columns=['label'])

    # Save final output
    ###############################################################
    # You can select between geopackage (gpkg) or feature class (fc)
    ###############################################################
    final_file = saving_dataframe(admin_shp, src_path, extension, 'ArcPro', gdf_final)

    if final_file == 'error':
        print ('----------------------------------------------------------------\n'
               'Nothing has been saved\n'
               'The extension needs to be xlsx or gpkg \n'
               '----------------------------------------------------------------')
    else:
        print ('----------------------------------------------------------------\n'
               f'The file has been created at: {Path(src_path/"2_Active_Data/201_3www/")}\n'
               f'The file name is {final_file}\n'
               '----------------------------------------------------------------')

print('----------------------------\n'
      'Functions have been set\n'
      '----------------------------')

### 3. Process

Quick check of the variables set up by the user before running the process

In [ ]:
check_var_setup(src_path, excel_file, iconDict)

#### 3.1. Read Workbook

In [ ]:
main_data = pd.read_excel(Path(src_path / f'2_Active_Data/201_3www/{excel_file}'), index_col=None, sheet_name=main_tab)
acro_data = pd.read_excel(Path(src_path / f'2_Active_Data/201_3www/{excel_file}'), index_col=None, sheet_name=acron_tab)

# Set up the acronym data as a dictionary
# convert acro_data to dict
'''this will enable the map function is just faster than merge'''
acro_dict = dict(acro_data.values)

print('----------------------------------------------------\n'
      'This is how the acronyms dictionary has been set:\n'
      '----------------------------------------------------')
pprint(acro_dict)

#### 3.2. Fixing the data

The script can be run without having to use ```df_replace``` function. However, if you have to do several changes in your spreadsheet, you might want to make use of this function.
An example of using the function is as follows:

***Putting the right name in sectors***
```python 
df = df_replace(df, "SECTOR / CLUSTER", 'Coordination', 'Coordinación y gestión de albergues')
``` 

In this example the function is replacing the value ***Coordinación y gestión de albergues*** for ***Coordination*** in the ***SECTOR / CLUSTER*** column. This can be done for any column or any value

##### 3.2.1. Adding an acronym column to geodataframe

In [ ]:
# 1. Adding the right acronym
main_data['org_acronym'] = main_data[ORGS].map(acro_dict)

# 2. Remove rows where the PCODE value is empty
main_data = remove_empty(main_data, pcod_group)
# main_data[pcod_group].replace('', np.nan, inplace=True)
# main_data = main_data[main_data[pcod_group].notna()]

# 3. Remove rows where the Organization value is empty 
main_data = main_data[main_data[ORGS].notna()]

# 4. Remove rows where the sector value is empty 
main_data = main_data[main_data[SECTOR].notna()]

# 5. Chage NaN back to empty to avoid errors
main_data = main_data.fillna('')

print('----------------------------------------------------\n'
      'This is an example of what the main data looks like:\n'
      '----------------------------------------------------')
main_data.head(2)

#### 3.3. Setting list of names and pcodes

This is going to be used as part of the label. In addition, this step provides some data checking. 
For the process to work the relation between admin name and pcode needs to be 1:1 i.e. one admin area needs to have one pcode. Sometimes the data will have different admin names for the same pcode. If that is the case then you can either fix it manually every time, or you can put the fix using the previous cell and the ```df_replace``` function

In [ ]:
# Set dataframe ready for process using grouping
df_group = main_data.groupby([pcod_group,STATUS, SECTOR, admn_group])['org_acronym'].apply(list).reset_index()

# Work only with Pcode and Adm name
'''This is needed to create a list of unique pair combination pcode - admn name.
At this stage there might be a duplication in pcode as we are only checking for 
duplication in the pcode-admn combination'''
df_duplicate_check = df_group[[pcod_group, admn_group]].copy()
df_duplicate_check = df_duplicate_check.groupby([pcod_group, admn_group]).first().reset_index()

# Check if there is still duplication in pcode values
'''This will return True if the value is duplicated after the first occurrance
i.e. the first time a value appears is set as False, if appears again then the 
new finding is set as True. If there are True values then there is an error
in the data that needs fixing i.e. one pcode value has been used in more than
one admn area'''
df_duplicate_check['pcode_duplicated'] = df_duplicate_check.duplicated(subset=[pcod_group])
# Check for error in the data
if (df_duplicate_check.loc[df_duplicate_check['pcode_duplicated'] == True]).empty is False:
    # save dataframe with errors
    saving_dataframe(admin_shp, src_path, 'xlsx', 'errors', df_duplicate_check)
    print ('there is an error in the raw data.\n' 
           f'You might have more than one name for {admn_group} with the same {pcod_group}.\n'
           f'I have saved an excel file with the problematic {admn_group} and {pcod_group} here:\n'
           f'2_Active_Data/201_3www/ \n'
           '-----------------------------------------------\n'
           'Please DO NOT carry as the output will be wrong\n'
           '-----------------------------------------------')
else:
    lst_adm_pcodes = df_duplicate_check[pcod_group].to_list()
    lst_adm_names = df_duplicate_check[admn_group].to_list()
    print('------------------------------------------------\n'
          f'There is no issue with the data. The {admn_group} and {pcod_group} match as expected\n'
          '-----------------------------------------------\n'
          'Please CARRY ON running the notebook\n'
          '-----------------------------------------------')
df_group.head(5)
    

#### 3.4. Get number of activites

This will count the number of activities as per the map label and will put the information in a list of list ready for next process


In [ ]:
# Working with the PCodes
lst_label = []
for count, pcode in enumerate(lst_adm_pcodes):
      subset = df_group[df_group[pcod_group] == pcode]
      lst_sector = list(set(list(subset[SECTOR])))
      
      # Getting info for each pcode
      for sector in lst_sector:
            df_sector = subset[subset[SECTOR] == sector]
            df_sector_inProg = df_sector[df_sector[STATUS] == ongoing]
            df_sector_planned = df_sector[df_sector[STATUS] == planned]
            
             # Set totals
            if df_sector.empty:
                  total_activites = 0
                  organisations = ''
            else:
                  total_activites = np.concatenate(df_sector["org_acronym"].values).size
                  organisations = ', '.join([elem for elem in np.unique(np.concatenate(df_sector["org_acronym"].values))])
            
            # In progress
            if df_sector_inProg.empty:
                  total_inProg = 0
            else:
                  total_inProg = np.concatenate(df_sector_inProg["org_acronym"].values).size
           
            # Planning
            if df_sector_planned.empty:
                  total_planned = 0
            else:
                  total_planned = np.concatenate(df_sector_planned["org_acronym"].values).size
            
            # Append values to list for label
            lst_label.append([pcode, lst_adm_names[count],sector, total_activites, total_inProg,total_planned,organisations])

print('----------------------------------------------------------------\n'
      'This is an example of what the list of list looks like:\n'
      '----------------------------------------------------------------')
pprint(f'{lst_label[0:5]}')

#### 3.5. Create dataframe ready to build label
The names of the columns here do not need changing at any point. For here onwards nothing should be changed


In [ ]:
df_labels = pd.DataFrame(lst_label, columns=[pcod_group, 
                                             admn_group,
                                             'sector', 
                                             'total_activites',
                                             'total_inProg',
                                             'total_planned',
                                             'organisations'])
                         
df_labels = df_labels.loc[(df_labels[['total_activites']] != 0).all(axis=1)]

# Add breaks every forth organization
'''This is needed to make the label in ArcPro nicer
It uses the add_break function'''  
  
df_labels['organisations'] = df_labels.apply(add_break, axis=1)

print('-----------------------------------------------------------------------------\n'
      'This is an example of what the dataframe to be used for labelling looks like:\n'
      '-----------------------------------------------------------------------------')

df_labels.head(5)
        

#### 3.6. Building the label for each activity
Here we are building the label for each sector i.e each row will be the label for that sector. The dataframe will show the same ADMN as many times as sectors in that ADMN

In [ ]:
# Build the label
# OCHA Font
iconFontName = 'Humanitarian-Icons'

# Set the string needed for each part of the label - ArcPro
close_activity = '</CLR></FNT>'
sector_format = f"\n<FNT name='{iconFontName}' size='10'><CLR red='60' green='96' blue='173'>"
total_format = f"<FNT name='calibri' size='10'><CLR red='255' green='0' blue='0'>"
inPro_format = f"<FNT name='calibri' size='10'><CLR red='0' green='0' blue='255'>"
plann_format = f"<FNT name='calibri' size='10'><CLR red='255' green='170' blue='0'>"

# Here we are applying apply function which performs the required action on each row
df_labels['label'] = df_labels.apply(lambda x: label_create(x, 
                                                            sector_format,
                                                            close_activity,
                                                            total_format,
                                                            inPro_format,
                                                            plann_format), 
                                     axis =1)

# Set the string needed for each part of the label - QGIS
close_activity = "</span>"
sector_format = "'<p><br /><span style=" + '"font-family: Humanitarian-Icons;' + 'color:rgb(60,96,173)";>'
total_format = "<span style=" + '"color:rgb(255,0,0); font-size:18px";>' + "'"
inPro_format = "<span style=" + '"color:rgb(0,0,255); font-size:18px";>' + "'"
plann_format = "<span style=" + '"color:rgb(255,170,0); font-size:18px";>' + "'"

# Here we are applying apply function which performs the required action on each row
df_labels['label_qg'] = df_labels.apply(lambda x: label_create_qgis(x, 
                                                                    sector_format,
                                                                    close_activity,
                                                                    total_format,
                                                                    inPro_format,
                                                                    plann_format), 
                                        axis =1)


print('-----------------------------------------------------------------------------\n'
      'This is an example of what the dataframe to be used for labelling looks like:\n'
      '-----------------------------------------------------------------------------')
df_labels.head(5)

#### 3.7. Create final dataframe
Here we will group the data by admn and PCode to build the complete label, that is all th sectors into one

In [ ]:
# Group all the labels by PCODE
'''They will need to go into a list'''
# ArcPro
df_labels_pro = df_labels.groupby([pcod_group, admn_group])['label'].apply(list).reset_index()
# Here we are applying apply function which performs the required action on each row
df_labels_pro['pro_label'] = df_labels_pro.apply(lambda x: label_gis(x, 'label', 'ArcPro'), axis =1)

# QGIS
df_labels_qgs = df_labels.groupby([pcod_group, admn_group])['label_qg'].apply(list).reset_index()
df_labels_qgs['qgis_label'] = df_labels_qgs.apply(lambda x: label_gis(x, 'label_qg', 'QGIS'), axis =1)

# Merge ArcPro and QGIS into one dataframe with label names
df_labels_final = df_labels_pro[[pcod_group, 'pro_label']].merge(df_labels_qgs[[pcod_group, 'qgis_label']], on=pcod_group, how='inner')


print('-------------------------------------------------------------------------\n'
      'This is an example of what the dataframe with the final label looks like:\n'
      '-------------------------------------------------------------------------')


df_labels_final.head(2)

#### 3.8. Create the geopackage o feature class
Here the dataframe with the labels and the geodatrame (admn shapefile) are merged together (this works as a join). A geopackage or feature class needs to be created as the length of the label field will not work if it is a shapefile. That is the 255 character restriction. The geopackage will have the date added as a suffix to keep track of the version

In [ ]:
# Rename column in df_label_final
'''this renames the pcode column in the excel data to match
the admn shapefile ADM2_PCODE column name'''
pcode_field = f'ADM{admn_ref}_PCODE'
df_labels_final.columns = df_labels_final.columns.str.replace(pcod_group, pcode_field)

# Create output
create_output(src_path, admn_ref, df_labels_final, OUT_TYPE, pcode_field)